In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark dynamics
Statistical shape analysis 2019, exercise session 1

In [ ]:
%cd ..

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(2) # two landmarks, but this can be changed below
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.N.set_value(4) # number of landmarks
M.k_sigma.set_value(tensor(np.diag((.5,.5))))

q = tensor(np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten())
q1 = tensor(np.vstack((np.linspace(-.5,.5,M.N.eval()),1+np.zeros(M.N.eval()))).T.flatten())
v = tensor(np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten()) # velocity
p = M.flatf(q,v) # momentum
print("q = ", q)
print("p = ", p)

M.plot()
M.plotx(q,v,curve=True)
plt.show()

In [ ]:
# Hamiltonian dynamics
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
M.plot()
M.plotx(qs,v,curve=True)
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

# Boundary value problem

In [ ]:
# Logarithm map
from src.Riemannian import Log
Log.initialize(M,f=M.Exp_Hamiltonian)

q1 = tensor(np.vstack((np.linspace(-.5,.5,M.N.eval()),1+np.zeros(M.N.eval()))).T.flatten())
p_Log = M.Logf(q,q1)[0]
print(p_Log)
print(p)

qs = M.Exp_Hamiltoniantf(q,p_Log).T
M.plot()
M.plotx(qs,p_Log,curve=True)
plt.show()

# Lift to Diff(R^2)

In [ ]:
# plot grid deformation
from src.dynamics import jets
jets.initialize(M)

# set kernel width
M.k_sigma.set_value(tensor(np.diag((.5,.5)))) # try width e.g np.diag((.1,.1))
p = M.flatf(q,v) # momentum
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]

M.plot()
M.plotx(qs,v,curve=True)
(grid,Nx,Ny) = M.getGrid(-1.,1.,-1.,1.,xpts=50,ypts=50)
mgrid = M.Ham_advectf(grid,qs,ps)[1][-1]
M.plotGrid(mgrid,Nx,Ny,coloring=True)
plt.show()